In [44]:
#!value --name example01
467..114..
...*......
..35...633
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In [69]:
#!share --from value example01

type Part = {
    row:int
    head:int
    tail:int
    value:string
}
module Part =
    let create row head value =
        { row = row; head = head; tail = -1; value = string value }
    let add tail value part =
        { part with tail = tail; value = part.value + string value }

type Symbol = {
    value: char
    row:int
    col:int
}

type Row = {
    col: int
    last: Part option
    parts: Part seq
    symbols: Symbol seq
}
module Row =
    let parse num (row:string) =
        let result =
            row.Trim().ToCharArray()
            |> Seq.fold (fun (state:Row) it ->
                match it with
                | digit when Char.IsDigit(digit) ->
                    { state with 
                        col = state.col + 1
                        last = state.last |> Option.map( Part.add state.col digit ) |> Option.orElse( (Part.create num state.col digit |> Some) )
                    }
                | '.' -> 
                    { state with 
                        col = state.col + 1
                        last = None
                        parts = match state.last with Some part -> state.parts |> Seq.append [ part ] | None -> state.parts 
                    }
                | symbol -> 
                    { state  with 
                        col = state.col + 1
                        symbols = state.symbols |> Seq.append [ { col = state.col; row = num; value = symbol } ]
                    }
            ) { col = 0; last = None; parts = Seq.empty; symbols = Seq.empty }
        { result with 
            parts = match result.last with Some part -> result.parts |> Seq.append [ part ] | None -> result.parts } 


let parse lines =
    lines
    |> Seq.mapi Row.parse
    |> Seq.fold (fun (parts, symbols) row -> 
        ( parts |> Seq.append row.parts) , ( symbols |> Seq.append row.symbols )
    ) (Seq.empty, Seq.empty)

let filterValid (parts, symbols) =
    symbols
    |> Seq.map(fun symbol ->
        parts
        |> Seq.filter( fun (part:Part) ->
            match part with
            | { head = h; tail = t; row = r; value = _ } when r = symbol.row ->
                h = symbol.col + 1 || t = symbol.col - 1
            | { head = h; tail = t; row = r; value = _ } when r = symbol.row + 1 || r = symbol.row - 1 ->
                h <= symbol.col + 1 && t >= symbol.col - 1
            | _ -> false
        )
    )
    |> Seq.concat

example01.Split("\n") 
|> parse
|> filterValid
|> Seq.sumBy (fun part -> int part.value)

22837

In [70]:
let ResolutionFolder = __SOURCE_DIRECTORY__

File.ReadAllText(ResolutionFolder + "/input01.txt").Split("\n")
|> parse
|> filterValid
//|> Seq.map (fun part -> int part.value)
//|> Seq.sum

[ { row = 139
 head = 91
 tail = 93
 value = "687" }, { row = 139
 head = 66
 tail = 68
 value = "789" }, { row = 139
 head = 28
 tail = -1
 value = "6" }, { row = 139
 head = 21
 tail = 23
 value = "244" }, { row = 138
 head = 102
 tail = 104
 value = "829" }, { row = 138
 head = 95
 tail = 97
 value = "348" }, { row = 138
 head = 62
 tail = 64
 value = "591" }, { row = 138
 head = 36
 tail = 38
 value = "806" }, { row = 138
 head = 32
 tail = 33
 value = "57" }, { row = 138
 head = 11
 tail = 13
 value = "256" }, { row = 137
 head = 136
 tail = 138
 value = "259" }, { row = 137
 head = 76
 tail = 78
 value = "737" }, { row = 137
 head = 72
 tail = 74
 value = "630" }, { row = 137
 head = 43
 tail = 45
 value = "363" }, { row = 137
 head = 20
 tail = 22
 value = "702" }, { row = 137
 head = 5
 tail = 6
 value = "86" }, { row = 136
 head = 127
 tail = 129
 value = "555" }, { row = 136
 head = 119
 tail = 121
 value = "767" }, { row = 136
 head = 107
 tail = 109
 value = "836" }, { row = 136
 head = 102
 tail = 104
 value = "238" } ... (more) ] f Microsoft.FSharp.Core.CompilerServices.RuntimeHelpers+mkConcatSeq@354[System.Collections.Generic.IEnumerable`1[FSI_0138+Part],FSI_0138+Part] sources [ [ { row = 139
 head = 91
 tail = 93
 value = "687" }, { row = 139
 head = 66
 tail = 68
 value = "789" }, { row = 139
 head = 28
 tail = -1
 value = "6" }, { row = 139
 head = 21
 tail = 23
 value = "244" }, { row = 138
 head = 102
 tail = 104
 value = "829" }, { row = 138
 head = 95
 tail = 97
 value = "348" }, { row = 138
 head = 62
 tail = 64
 value = "591" }, { row = 138
 head = 36
 tail = 38
 value = "806" }, { row = 138
 head = 32
 tail = 33
 value = "57" }, { row = 138
 head = 11
 tail = 13
 value = "256" }, { row = 137
 head = 136
 tail = 138
 value = "259" }, { row = 137
 head = 76
 tail = 78
 value = "737" }, { row = 137
 head = 72
 tail = 74
 value = "630" }, { row = 137
 head = 43
 tail = 45
 value = "363" }, { row = 137
 head = 20
 tail = 22
 value = "702" }, { row = 137
 head = 5
 tail = 6
 value = "86" }, { row = 136
 head = 127
 tail = 129
 value = "555" }, { row = 136
 head = 119
 tail = 121
 value = "767" }, { row = 136
 head = 107
 tail = 109
 value = "836" }, { row = 136
 head = 102
 tail = 104
 value = "238" } ... (more) ], [ { row = 139
 head = 66
 tail = 68
 value = "789" }, { row = 139
 head = 28
 tail = -1
 value = "6" }, { row = 139
 head = 21
 tail = 23
 value = "244" }, { row = 138
 head = 125
 tail = 127
 value = "460" }, { row = 138
 head = 102
 tail = 104
 value = "829" }, { row = 138
 head = 62
 tail = 64
 value = "591" }, { row = 138
 head = 36
 tail = 38
 value = "806" }, { row = 138
 head = 32
 tail = 33
 value = "57" }, { row = 138
 head = 11
 tail = 13
 value = "256" }, { row = 137
 head = 136
 tail = 138
 value = "259" }, { row = 137
 head = 76
 tail = 78
 value = "737" }, { row = 137
 head = 72
 tail = 74
 value = "630" }, { row = 137
 head = 43
 tail = 45
 value = "363" }, { row = 137
 head = 20
 tail = 22
 value = "702" }, { row = 137
 head = 5
 tail = 6
 value = "86" }, { row = 136
 head = 127
 tail = 129
 value = "555" }, { row = 136
 head = 119
 tail = 121
 value = "767" }, { row = 136
 head = 107
 tail = 109
 value = "836" }, { row = 136
 head = 102
 tail = 104
 value = "238" }, { row = 136
 head = 98
 tail = 100
 value = "955" } ... (more) ], [ { row = 139
 head = 91
 tail = 93
 value = "687" }, { row = 139
 head = 28
 tail = -1
 value = "6" }, { row = 139
 head = 21
 tail = 23
 value = "244" }, { row = 138
 head = 125
 tail = 127
 value = "460" }, { row = 138
 head = 102
 tail = 104
 value = "829" }, { row = 138
 head = 95
 tail = 97
 value = "348" }, { row = 138
 head = 36
 tail = 38
 value = "806" }, { row = 138
 head = 32
 tail = 33
 value = "57" }, { row = 138
 head = 11
 tail = 13
 value = "256" }, { row = 137
 head = 136
 tail = 138
 value = "259" }, { row = 137
 head = 76
 tail = 78
 value = "737" }, { row = 137
 head = 72
 tail = 74
 value = "630" }, { row = 137
 head = 43
 tail = 45